In [82]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [83]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [84]:
def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de problemas
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de usuarios que ha hecho cada problema
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado

In [85]:
def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    

In [86]:
def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo

In [87]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2015-07-01 00:00:00")

print(len(training_set))

# obtengo los nodos del grafo:
nodes = training_set.problem_id.unique()


# creo un diccionario que va a tener a los problemas como keys y los valores seran los
# usuarios que han hecho ese problema
grouped = training_set.groupby('problem_id')['user_id'].apply(list)

print(len(nodes))
# print(grouped)


4712
169


In [88]:
print(training_set)

      problem_id  user_id status       submissionDate
0             10        5     AC  2014-02-17 15:27:07
1              2        6     AC  2014-02-17 15:39:17
2              2        9     AC  2014-02-18 00:30:14
3             10        9     AC  2014-02-18 00:34:46
4              4        9     AC  2014-02-18 00:50:28
5              6        9     AC  2014-02-18 00:52:11
6             13        9     AC  2014-02-18 00:53:40
9             15        8     AC  2014-02-19 19:58:03
10             4        8     AC  2014-02-20 14:23:30
11            39       16     AC  2014-02-20 15:44:33
12            39       17     AC  2014-02-20 16:53:34
13            13       12     AC  2014-02-21 11:08:38
15            33       12     AC  2014-02-21 11:58:32
16            39       12     AC  2014-02-21 12:05:49
17            44       12     AC  2014-02-21 12:52:44
18            44       15     AC  2014-02-21 14:51:52
19            44       22     AC  2014-02-21 14:58:55
21            19        8   

In [89]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2015-07-01 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
6933          117      725     WA  2015-07-01 09:17:38
6934          122      725    RTE  2015-07-01 11:09:50
6935            2     1665     CE  2015-07-01 13:15:06
6936          340     1504     WA  2015-07-01 16:12:24
6937          443     1735     TL  2015-07-01 18:55:30
6938           39     1737     AC  2015-07-01 20:36:18
6939          134     1317     AC  2015-07-02 13:37:18
6940          150     1504     AC  2015-07-02 19:43:11
6941          183     1504     AC  2015-07-02 20:37:12
6942           39     1736     AC  2015-07-03 00:00:53
6943          195     1504     AC  2015-07-03 03:53:16
6944          508     1504     AC  2015-07-03 04:19:22
6945            2     1504     AC  2015-07-03 04:29:37
6946          174     1504     AC  2015-07-03 17:16:06
6947           35      103     AC  2015-07-03 18:10:52
6948          181     1504     AC  2015-07-03 22:08:51
6949            8     1736     AC  2015-07-04 15:46:08
6950      

In [90]:

# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

4443


In [91]:
def apply_cn(row, graph):
    """
        Funcion que devuelve el numero de vecinos en comun de esos dos nodos
    """
    return len(list(nx.common_neighbors(graph, row['one'], row['two'])))

def create_cn_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de cn para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_cn = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_cn['cn'] = dataFrame_cn.apply (lambda row: apply_cn(row, graph), axis=1)


    return dataFrame_cn


In [92]:
cn_df = create_cn_data(graph, nodes)
print(cn_df)

       one  two  cn
0       10    2  48
1       10    4  43
2       10    6  38
3       10   13  42
4       10   15  46
5       10   39  48
6       10   33  44
7       10   44  45
8       10   19  30
9       10   27  24
10      10   60  23
11      10  100  45
12      10   93   0
13      10   53  41
14      10   51  41
15      10   49  47
16      10   70  44
17      10   62  47
18      10   35  36
19      10   81   5
20      10    8  33
21      10   17   8
22      10   23  23
23      10   25   0
24      10   86  15
25      10   29  44
26      10   31   0
27      10   47   0
28      10   65   0
29      10   73  29
...    ...  ...  ..
14166  510  509  31
14167  510  507   0
14168  511  504   0
14169  511  503  32
14170  511  505  12
14171  511  506   0
14172  511  502  19
14173  511  509  34
14174  511  507   0
14175  504  503   0
14176  504  505   0
14177  504  506   0
14178  504  502   0
14179  504  509   0
14180  504  507   0
14181  503  505  13
14182  503  506   0
14183  503  502  21


In [93]:
# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user = training_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped_user.index, 'list_problem_id':grouped_user.values})

print(df_users)

                                       list_problem_id  user_id
0                                        [10, 76, 489]        5
1                                                  [2]        6
2                     [39, 44, 76, 105, 109, 114, 117]        7
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9
5                                            [33, 109]       11
6    [13, 33, 39, 44, 100, 51, 49, 70, 81, 53, 130,...       12
7                                            [44, 187]       15
8                                                 [39]       16
9                                             [39, 86]       17
10                                                [44]       22
11                                           [100, 13]       24
12                 [258, 141, 309, 310, 390, 187, 510]       25
13                                               [100]       28
14                                      

In [94]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    
    

In [95]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)


      user_id  len_training  len_evaluation
0           5             3               2
1           6             1               1
2           9             9               5
3           8            40               8
4          16             1               0
5          17             2               2
6          12            12               0
7          18             0              13
8          15             2               2
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            40               2
16         33             7               2
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [96]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))


      user_id  len_training  len_evaluation
2           9             9               5
3           8            40               8
29         60             6              27
118        25             7              33
261       130            62              34
274       382             5               6
285       206           111              38
286        62           112              32
299       414             9             103
310       437           134              13
316       443            41              22
317       448            31              33
333       381            13              10
348       103           109              48
383       542             8              13
395       431            11               8
483       689             5              41
597       254            11              12
601       864            14               8
619       912             7              17
621       840            26              19
633       935            88     

In [97]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users)

                                       list_problem_id  user_id
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9
12                 [258, 141, 309, 310, 390, 187, 510]       25
37                            [100, 62, 2, 57, 53, 23]       60
38   [178, 251, 224, 159, 191, 183, 166, 139, 256, ...       62
69   [33, 51, 60, 70, 83, 134, 181, 187, 191, 206, ...      103
71        [10, 191, 228, 441, 437, 438, 436, 109, 122]      105
88   [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
135  [53, 119, 251, 256, 224, 253, 181, 155, 191, 1...      206
162  [259, 141, 250, 254, 70, 255, 195, 159, 252, 3...      254
224  [259, 235, 226, 51, 119, 316, 258, 352, 272, 2...      381
225                          [119, 262, 264, 114, 388]      382
237         [213, 2, 109, 114, 10, 436, 438, 437, 404]      414
245  [39, 259, 258, 203, 251, 60, 49, 195, 6, 159, 70]      431
248  [250, 124, 249, 100, 4, 122, 57, 30

In [98]:
# creo el nuevo dataframe con los resultados 
column_problem_recomend = {'problem_id': nodes}
dataframe_problem_recomend = pd.DataFrame.from_dict(column_problem_recomend)

print(dataframe_problem_recomend)

     problem_id
0            10
1             2
2             4
3             6
4            13
5            15
6            39
7            33
8            44
9            19
10           27
11           60
12          100
13           93
14           53
15           51
16           49
17           70
18           62
19           35
20           81
21            8
22           17
23           23
24           25
25           86
26           29
27           31
28           47
29           65
..          ...
139         444
140         437
141         438
142         443
143         445
144         442
145         440
146         446
147         465
148         471
149         468
150         470
151         469
152         467
153         489
154         485
155         486
156         487
157         484
158         488
159         508
160         510
161         511
162         504
163         503
164         505
165         506
166         502
167         509
168         507

[169 ro

In [99]:
def getSimilarProblems(row, cn_df):
    """
        Funcion que devuelve la lista de los problemas similares del problema de la fila: aquellos que tienen un valor cn
        mayor a cero
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = cn_df[cn_df['one'] == row['problem_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['cn'] > 0]
    column_result_two_tmp = cn_df[cn_df['two'] == row['problem_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['cn'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_cn = list_one + list_two
    
    # print(list_cn)
    
    if list_cn == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_cn = graph.nodes()
        list_cn.remove(row['problem_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_cn

In [100]:
dataframe_problem_recomend['neighbors'] = dataframe_problem_recomend.apply (lambda row: getSimilarProblems(row, cn_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_problem_recomend)

     problem_id                                          neighbors
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...
13           93  [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10

In [101]:
print(grouped)
# convierto la serie en un dataframe
df_users_recommend = pd.DataFrame({'problem_id':grouped.index, 'list_user_id':grouped.values})
print(df_users_recommend)

problem_id
2      [6, 9, 33, 67, 56, 84, 35, 60, 112, 124, 106, ...
4      [9, 8, 35, 84, 126, 44, 39, 313, 42, 437, 130,...
6      [9, 46, 35, 67, 89, 84, 137, 152, 231, 221, 88...
8      [35, 52, 133, 129, 130, 437, 511, 605, 935, 67...
10     [5, 9, 35, 133, 129, 80, 137, 247, 437, 103, 5...
13     [9, 12, 59, 35, 67, 109, 54, 24, 55, 106, 136,...
15     [8, 35, 67, 55, 133, 129, 84, 167, 205, 231, 2...
17                              [35, 133, 129, 437, 103]
19     [8, 35, 44, 205, 437, 527, 448, 1063, 206, 62,...
23     [35, 67, 60, 437, 206, 62, 103, 864, 1001, 113...
25                                             [35, 103]
27     [8, 35, 67, 167, 39, 440, 448, 437, 130, 488, ...
29     [35, 133, 86, 112, 42, 232, 248, 247, 314, 320...
31                                        [35, 437, 448]
33     [12, 59, 35, 84, 111, 96, 152, 126, 155, 30, 1...
35     [64, 35, 214, 120, 236, 82, 409, 390, 175, 249...
39     [16, 17, 12, 7, 31, 40, 33, 66, 68, 71, 69, 70...
44     [12, 15, 22, 

In [102]:
def cn_value(one, two, graph):
    """
        Funcion que devuelve el numero de vecinos en comun de esos dos nodos
    """
    return len(list(nx.common_neighbors(graph, one, two)))

In [103]:
def getWeights(row, graph):
    """
        Funcion que calcula la suma de todos los valores de cn de todos los nodos con los que tiene cn
    """
    
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # print(neighbors_list)
    
    problem = row['problem_id']
    suma = 0
    
    for elem in neighbors_list:
        # print(suma)
        # print(graph[user][elem]['weight'])
        suma = suma + cn_value(problem, elem, graph)
        
    return suma

In [104]:
def getPonderaciones(row, graph):
    """
        Funcion que calcula la ponderacion para cada problema vecino del de la fila
        Calculo la ponderacion diviendo el peso del enlace que enlaza cada problema con problem_id con la suma 
        total de los pesos
    """
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # obtengo la suma de pesos de los enlaces de esa lista
    total_weight = row['total_weight']
    
    # obtengo el id del usuario al que quiero recomendar
    problem = row['problem_id']
    
    lista_ponderaciones = list()
    
    for elem in neighbors_list:
        
        # obtengo el peso del enlace
        peso_enlace = cn_value(problem, elem, graph)
        
        if total_weight == 0:
            ponderacion = 0
        else:
            # hago la ponderacion
            ponderacion = peso_enlace/total_weight
        
        lista_ponderaciones.append(ponderacion)
        
    return lista_ponderaciones
    

In [105]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_problem_recomend['total_weight'] = dataframe_problem_recomend.apply (lambda row: getWeights(row, graph), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [106]:
# para cada usuario a "recomendar" para cada problema miro el numero de apariciones en su lista

dataframe_problem_recomend['score'] = dataframe_problem_recomend.apply (lambda row: getPonderaciones(row, graph), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [107]:
def getUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de la concatenacion de listas de usuarios que han
        realizado los problemas similares al de la fila. Además eliminara los usuarios que ya hayan realizado el problema.
    """
    
    # obtengo la lista de usuarios que han hecho el problema en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # print(list_problems_user)

    # lista resultante de la concatenacion de las listas de usuarios de los problemas similares
    list_result = list()
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de usuarios que han hecho el problema similar
        list_problems_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_user_id'])[0]
        
        # print("----------")
        #print(lista_problemas_comprobar)
        
        # aqui hago el filtro para que no se incluyan los usuarios que ya ha hechon el problema
        list_users = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora concateno el resultado
        list_result = list_result + list_users
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [108]:
# ahora para cada problema, hacer una lista de los usuarios similares a los que han realizado ese problema, 
# que no sean usuarios que han realizado ya ese problema
dataframe_problem_recomend['list_users'] = dataframe_problem_recomend.apply (lambda row: getUsersFromSimilarProblems(row, df_users_recommend), axis=1)
print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [109]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['list_users'] if not (x in conjunto_vacio or function_add(x))]

In [110]:

# tengo que contar el numero de apariciones de cada usuario en la lista de usuarios

# voy a sacar primero una lista sin repeticiones
# ahora voy a crear una nueva columna que contenga la lista de usuarios sin repeticiones
dataframe_problem_recomend['lista_users_unique'] = dataframe_problem_recomend.apply(lambda row: delRepetitions(row), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [111]:
# elimino la columna de la lista de problemas (la que tiene repeticiones), ya que no interesa
del dataframe_problem_recomend['list_users']
# elimino la columna de total_weight, ya que no interesa
del dataframe_problem_recomend['total_weight']

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [112]:
def getListUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de listas la concatenacion de listas de usuarios que han
        hecho los problemas similares a ese. Además eliminara los usuarios que ya hayan hecho el problema
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # lista resultante 
    list_result = list(list())
    
    # obtengo la longitud de la lista de vecinos de ese problema
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de problemas vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_users_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_users_df['list_user_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora incluyo la lista en la lista
        list_result.append(list_problems)
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [113]:
dataframe_problem_recomend['lista_users_por_problema'] = dataframe_problem_recomend.apply(lambda row: getListUsersFromSimilarProblems(row, df_users_recommend), axis=1)

print(dataframe_problem_recomend)


     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [114]:
# ahora voy a separar cada problema_user para hacer la cuenta
# creo un nuevo dataframe que agrupa por el problema con su posible usuario
df_separation = dataframe_problem_recomend.groupby(['problem_id']).lista_users_unique.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation.columns = ['problem_id', 'possible_user', ]

print(df_separation)

        problem_id  possible_user
0                2            126
1                2             39
2                2            313
3                2             42
4                2             62
5                2             46
6                2             89
7                2            152
8                2            221
9                2             88
10               2            194
11               2             90
12               2             99
13               2            155
14               2            542
15               2            896
16               2           1186
17               2            598
18               2            584
19               2            596
20               2            594
21               2            593
22               2            581
23               2            585
24               2            597
25               2            582
26               2            589
27               2            600
28            

In [115]:
def getScoring(row, dataframe_problem_recomend):
    """
        Funcion en la que por cada usuario-problema, hace el scoring para ese problema (para ese usuario)
        sumando todas las ponderaciones de ese problema para ese usuario
    """
     # obtengo la lista de listas de usuarios que han hecho los vecinos del problema
    list_problems_users_df = dataframe_problem_recomend[dataframe_problem_recomend['problem_id'] == row['problem_id']]
    list_user_per_problem = list(list_problems_users_df['lista_users_por_problema'])[0]
    
    # saco tambien la lista de scores por vecinos
    list_scores_per_problem = list(list_problems_users_df['score'])[0]
    
    # print(list_scores_per_user)
    
    # obtengo el usuario del que quiero calcular el scoring
    user = row['possible_user']
    # print(problem)
    
    suma = 0
    k = len(list_user_per_problem)
    
    # por cada lista de la lista, saco la ponderacion de su correspondiente usuario (el usuario que lo ha hecho)
    # si problem aparece en la lista, sumo esa ponderacion
    for i in range(0, k):
        lista_a_comprobar = list_user_per_problem[i]
        # print(lista_a_comprobar)
        if user in lista_a_comprobar:
            # print(list_scores_per_user[i])
            suma = suma + list_scores_per_problem[i]
    
    return suma

In [116]:
# ahora voy a calcular el scoring para cada usuario
df_separation['score'] = df_separation.apply(lambda row: getScoring(row, dataframe_problem_recomend), axis=1)

print(df_separation)

        problem_id  possible_user     score
0                2            126  0.073401
1                2             39  0.094968
2                2            313  0.010720
3                2             42  0.233195
4                2             62  0.909699
5                2             46  0.082230
6                2             89  0.007946
7                2            152  0.151091
8                2            221  0.033674
9                2             88  0.021062
10               2            194  0.007946
11               2             90  0.029638
12               2             99  0.034178
13               2            155  0.067222
14               2            542  0.070501
15               2            896  0.096734
16               2           1186  0.007946
17               2            598  0.096481
18               2            584  0.140749
19               2            596  0.209358
20               2            594  0.042502
21               2            59

In [117]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
df_separation = df_separation.sort_values(by=['possible_user', 'score'], ascending=False)
print(df_separation)

        problem_id  possible_user     score
19654           57           1733  0.011111
95200          311           1733  0.011091
41563          130           1733  0.010777
24051           73           1733  0.010755
49452          155           1733  0.010671
48555          150           1733  0.010652
51134          162           1733  0.010541
70361          237           1733  0.010525
11324           29           1733  0.010459
2083             6           1733  0.010453
17931           53           1733  0.010427
56334          183           1733  0.010426
6869            17           1733  0.010405
43283          136           1733  0.010401
27639           83           1733  0.010357
44166          139           1733  0.010339
33019           97           1733  0.010288
12388           33           1733  0.010273
25850           78           1733  0.010265
54589          178           1733  0.010239
4684            13           1733  0.010233
46695          145           173

In [118]:
# ahora tengo que hacer un nuevo dataframe con usuario y problema, y una lista de recommendation 

# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('possible_user')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

#print(df_recommend)

# vuelvo a crear la estructura de los datos para poder trabajar con ellos
df_recommend = pd.DataFrame({'user_id':df_recommend.index.values, 'problem_id':df_recommend['problem_id'].tolist()})
print(df_recommend)

                                            problem_id  user_id
0    [243, 250, 17, 86, 60, 97, 222, 27, 8, 206, 19...        5
1    [258, 49, 44, 29, 109, 15, 13, 39, 209, 254, 3...        6
2    [322, 282, 272, 365, 264, 124, 122, 128, 279, ...        7
3    [243, 322, 247, 17, 35, 73, 97, 119, 88, 29, 4...        8
4    [73, 250, 130, 83, 17, 27, 60, 243, 86, 35, 29...        9
5    [322, 282, 272, 39, 44, 365, 49, 2, 258, 62, 1...       11
6    [57, 83, 29, 4, 2, 109, 15, 6, 62, 183, 258, 2...       12
7    [33, 2, 109, 39, 49, 209, 258, 13, 70, 203, 25...       15
8    [272, 282, 365, 122, 279, 109, 62, 469, 264, 1...       16
9    [272, 282, 243, 250, 222, 264, 365, 109, 124, ...       17
10   [33, 2, 109, 258, 209, 13, 39, 183, 49, 254, 2...       22
11   [29, 122, 62, 39, 83, 109, 49, 4, 57, 2, 250, ...       24
12   [365, 469, 307, 254, 259, 134, 39, 62, 109, 20...       25
13   [365, 469, 62, 39, 29, 109, 254, 233, 134, 259...       28
14   [365, 469, 62, 39, 29, 109, 254, 23

In [119]:
def getProblems(row, df_recommend):
    """
        Funcion que copia los posibles problemas a recomendar para los usuarios a los que tengo que recomendar
    """
    
    # saco la lista de problemas 
    df_lista_problemas_original = df_recommend[df_recommend['user_id'] == row['user_id']]
    lista_problemas_original = list(df_lista_problemas_original['problem_id'])[0]
    
    return lista_problemas_original
    

In [120]:
# df_users # en este dataframe estan los usuarios a los que quiero recomendar

# ahora voy a copiar los problemas para los usuarios a los que tengo que recomendar
df_users['recommendation'] = df_users.apply (lambda row: getProblems(row, df_recommend), axis=1)

print(df_users)

                                       list_problem_id  user_id  \
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8   
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9   
12                 [258, 141, 309, 310, 390, 187, 510]       25   
37                            [100, 62, 2, 57, 53, 23]       60   
38   [178, 251, 224, 159, 191, 183, 166, 139, 256, ...       62   
69   [33, 51, 60, 70, 83, 134, 181, 187, 191, 206, ...      103   
71        [10, 191, 228, 441, 437, 438, 436, 109, 122]      105   
88   [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130   
135  [53, 119, 251, 256, 224, 253, 181, 155, 191, 1...      206   
162  [259, 141, 250, 254, 70, 255, 195, 159, 252, 3...      254   
224  [259, 235, 226, 51, 119, 316, 258, 352, 272, 2...      381   
225                          [119, 262, 264, 114, 388]      382   
237         [213, 2, 109, 114, 10, 436, 438, 437, 404]      414   
245  [39, 259, 258, 203, 251, 60, 49, 195, 6, 159, 70]      43

In [121]:
# elimino la columna de los problemas, ya que no interesa
del df_users['list_problem_id']

print(df_users)

     user_id                                     recommendation
3          8  [243, 322, 247, 17, 35, 73, 97, 119, 88, 29, 4...
4          9  [73, 250, 130, 83, 17, 27, 60, 243, 86, 35, 29...
12        25  [365, 469, 307, 254, 259, 134, 39, 62, 109, 20...
37        60  [122, 83, 130, 29, 73, 27, 13, 4, 39, 109, 6, ...
38        62  [469, 365, 2, 312, 446, 6, 250, 308, 150, 311,...
69       103  [282, 105, 209, 195, 117, 262, 235, 155, 141, ...
71       105  [322, 282, 272, 124, 487, 264, 279, 445, 262, ...
88       130  [15, 49, 251, 258, 253, 23, 254, 259, 325, 257...
135      206  [469, 365, 311, 51, 446, 162, 308, 178, 250, 6...
162      254  [365, 243, 258, 307, 256, 134, 251, 39, 2, 241...
224      381  [282, 469, 365, 39, 254, 256, 233, 307, 62, 38...
225      382  [272, 282, 322, 124, 279, 122, 128, 365, 117, ...
237      414  [322, 282, 122, 124, 272, 264, 128, 262, 487, ...
245      431  [272, 282, 109, 2, 256, 44, 254, 13, 33, 209, ...
248      437  [272, 13, 44, 70, 33, 390,

In [122]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    return row['recommendation'][:k]

In [123]:
k = 10

#print(list(range(1, k+1)))

rango = list(range(1, k+1))

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    #print(name_column)
    # ahora saco los k mejores problemas para cada usuario
    df_users[name_column] = df_users.apply(lambda row: getKrecomFinal(row, i), axis=1)

# he incluido una columna para cada cada k recomendacion 
print(df_users)

     user_id                                     recommendation  \
3          8  [243, 322, 247, 17, 35, 73, 97, 119, 88, 29, 4...   
4          9  [73, 250, 130, 83, 17, 27, 60, 243, 86, 35, 29...   
12        25  [365, 469, 307, 254, 259, 134, 39, 62, 109, 20...   
37        60  [122, 83, 130, 29, 73, 27, 13, 4, 39, 109, 6, ...   
38        62  [469, 365, 2, 312, 446, 6, 250, 308, 150, 311,...   
69       103  [282, 105, 209, 195, 117, 262, 235, 155, 141, ...   
71       105  [322, 282, 272, 124, 487, 264, 279, 445, 262, ...   
88       130  [15, 49, 251, 258, 253, 23, 254, 259, 325, 257...   
135      206  [469, 365, 311, 51, 446, 162, 308, 178, 250, 6...   
162      254  [365, 243, 258, 307, 256, 134, 251, 39, 2, 241...   
224      381  [282, 469, 365, 39, 254, 256, 233, 307, 62, 38...   
225      382  [272, 282, 322, 124, 279, 122, 128, 365, 117, ...   
237      414  [322, 282, 122, 124, 272, 264, 128, 262, 487, ...   
245      431  [272, 282, 109, 2, 256, 44, 254, 13, 33, 209, ..

In [124]:
# elimino las columnas que no me interesan
del df_users['recommendation']

print(df_users)

     user_id k_recommendation_1 k_recommendation_2 k_recommendation_3  \
3          8              [243]         [243, 322]    [243, 322, 247]   
4          9               [73]          [73, 250]     [73, 250, 130]   
12        25              [365]         [365, 469]    [365, 469, 307]   
37        60              [122]          [122, 83]     [122, 83, 130]   
38        62              [469]         [469, 365]      [469, 365, 2]   
69       103              [282]         [282, 105]    [282, 105, 209]   
71       105              [322]         [322, 282]    [322, 282, 272]   
88       130               [15]           [15, 49]      [15, 49, 251]   
135      206              [469]         [469, 365]    [469, 365, 311]   
162      254              [365]         [365, 243]    [365, 243, 258]   
224      381              [282]         [282, 469]    [282, 469, 365]   
225      382              [272]         [272, 282]    [272, 282, 322]   
237      414              [322]         [322, 282] 

In [125]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
2              [445, 436, 437, 441, 442, 443, 444, 62]        8
3                             [83, 109, 136, 141, 254]        9
9    [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...       25
16   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...       60
17   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...       62
21   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...      103
22   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...      105
24   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...      130
30   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...      206
34   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...      254
43    [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]      381
44                              [33, 8, 465, 6, 2, 13]      382
46   [239, 62, 187, 206, 313, 316, 504, 465, 191, 2...      414
50             [253, 33, 252, 443, 444, 441, 404, 503]      431
51   [489, 181, 390, 393, 397, 437, 486,

In [126]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems}

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    list_recom_problems = list()
    list_recom_problems = df_users[name_column].tolist()
    
    name_column_metric = 'recom_problems_' + str(i)
    set_df_metric[name_column_metric] = list_recom_problems

# he generado un nuevo dataframe con todas las recomendaciones (desde 1 a k) y los problemas realizados en 
# el evaluation set para cada usuario al que quiero recomendar

metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems recom_problems_1  \
0             [445, 436, 437, 441, 442, 443, 444, 62]            [243]   
1                            [83, 109, 136, 141, 254]             [73]   
2   [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...            [365]   
3   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...            [122]   
4   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...            [469]   
5   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...            [282]   
6   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...            [322]   
7   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...             [15]   
8   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...            [469]   
9   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...            [365]   
10   [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]            [282]   
11                             [33, 8, 465, 6, 2, 13]            [272]   
12  [239, 62, 187, 206, 313, 316, 504,

In [127]:
def one_hit(row, i):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [128]:
def mrr(row, i): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row[name_column])) and (encontrado == False):
            if row[name_column][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [129]:
def precision(row, i):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row[name_column]))

In [130]:
def recall(row, i):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [131]:
def f1(row, i):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    name_column_prec = 'precision_' + str(i)
    name_column_rec = 'recall_' + str(i)
    denominador = row[name_column_prec] + row[name_column_rec]
    
    if denominador == 0:
        return 0
    else:
        return (2 * row[name_column_prec] * row[name_column_rec]) / denominador

In [132]:
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    # ahora voy a calcular una metrica para cada usuario
    metric_df[name_one_hit] = metric_df.apply(lambda row: one_hit(row, i), axis=1)
    metric_df[name_mrr] = metric_df.apply(lambda row: mrr(row, i), axis=1)
    metric_df[name_precision] = metric_df.apply(lambda row: precision(row, i), axis=1)
    metric_df[name_recall] = metric_df.apply(lambda row: recall(row, i), axis=1)
    metric_df[name_f1] = metric_df.apply(lambda row: f1(row, i), axis=1)

print(metric_df)

                                        eval_problems recom_problems_1  \
0             [445, 436, 437, 441, 442, 443, 444, 62]            [243]   
1                            [83, 109, 136, 141, 254]             [73]   
2   [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...            [365]   
3   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...            [122]   
4   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...            [469]   
5   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...            [282]   
6   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...            [322]   
7   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...             [15]   
8   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...            [469]   
9   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...            [365]   
10   [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]            [282]   
11                             [33, 8, 465, 6, 2, 13]            [272]   
12  [239, 62, 187, 206, 313, 316, 504,

In [133]:


f = open("C:/hlocal/TFM/vot_simple_cn", 'a')

# calculo la media de las metricas
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    result_one_hit = metric_df[name_one_hit].mean()
    result_precision = metric_df[name_precision].mean()
    result_mrr = metric_df[name_mrr].mean()
    result_recall = metric_df[name_recall].mean()
    result_f1 = metric_df[name_f1].mean()

    # lo muestro por consola
    
    print(i)
    print("###########")
    print("One hit ----------")
    print(result_one_hit)
    print("Precision ----------")
    print(result_precision)
    print("Mrr  ----------")
    print(result_mrr)
    print("Recall  ----------")
    print(result_recall)
    print("F1  ----------")
    print(result_f1)
    
    
    f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 

    
f.close()    
    

1
###########
One hit ----------
0.14285714285714285
Precision ----------
0.14285714285714285
Mrr  ----------
0.14285714285714285
Recall  ----------
0.006036432867586677
F1  ----------
0.011457390838108641
2
###########
One hit ----------
0.30952380952380953
Precision ----------
0.19047619047619047
Mrr  ----------
0.2261904761904762
Recall  ----------
0.01693300227302912
F1  ----------
0.029603456165234188
3
###########
One hit ----------
0.4523809523809524
Precision ----------
0.23015873015873023
Mrr  ----------
0.27380952380952384
Recall  ----------
0.02970347253331188
F1  ----------
0.04995705376049029
4
###########
One hit ----------
0.5714285714285714
Precision ----------
0.25
Mrr  ----------
0.3035714285714286
Recall  ----------
0.04628401676418468
F1  ----------
0.07192956411984978
5
###########
One hit ----------
0.5952380952380952
Precision ----------
0.24285714285714283
Mrr  ----------
0.30833333333333335
Recall  ----------
0.05444614471412278
F1  ----------
0.081604374148651